# Final notebook with experiments for eye reconstruction and manipulation.

After training a decent autencoder over the ssh server, we began experimenting with different methods of changing the color of the eye, and manipulating the latent vectors or embeddings of te images.


In [ ]:

# USE THOS
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Necessary imports

Install commands

The line where the decoder saved state is loaded (decoder = load_model(filepath, custom_objects={'Conv2DTranspose': tf.keras.layers.Conv2DTranspose})) will not work unless the tensorflow version is 2.11.0.

Once ran, you will need to restart the session. You will not need to run the cell to install tensorflow v2.11.0 again. You may need to install the other libraries.

In [ ]:
!pip install tensorflow==2.11.0

In [ ]:
!pip install Pillow

In [ ]:
!pip install colormath

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
!pip install imageio

In [ ]:
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, BatchNormalization
from tensorflow.keras.models import Model, load_model
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
from scipy import interpolate
from sklearn.manifold import TSNE
from sklearn.metrics import mean_squared_error
import seaborn as sns
from PIL import Image
import gradio as gr
import imageio
import umap.umap_ as umap
from sklearn.cluster import KMeans
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

# USE THOS

# Loading Encoder/Data

This loads the saved state of the first half of the autencoder, which is the encoder. Over the remote server, the UBIRIS images are passed into the encoder, and the result is data and vectors representing each image.

In [ ]:
# USE THOS

IMG_SIZE = 128
LATENT_DIM = 32


image1 = cv2.imread('/content/drive/MyDrive/ML final project/our_eyes/IMG_1636.JPG')
image2 = cv2.imread('/content/drive/MyDrive/ML final project/our_eyes/IMG_1642.JPG')
image3 = cv2.imread('/content/drive/MyDrive/ML final project/our_eyes/IMG_1656.JPG')


"""
# Load the saved latent vectors and encoder model (FULL)
filepath = r'/content/drive/MyDrive/ML final project/Trained_models/encoder/encoder_model1.5.h5'
latent_vectors = np.load('/content/drive/MyDrive/ML final project/Trained_models/encoder/latent_vectors1.5.npy')
image_data = np.load('/content/drive/MyDrive/ML final project/Trained_models/encoder/image_data1.5.npy')
encoder = load_model(filepath)
#encoder = tf.keras.layers.TFSMLayer('encoder_model', call_endpoint='serving_default')
#image_data = np.load('/content/drive/MyDrive/ML final project/Trained_models/encoder/encoder_filtered/image_data8.npy')
"""

"\n# Load the saved latent vectors and encoder model (FULL)\nfilepath = r'/content/drive/MyDrive/ML final project/Trained_models/encoder/encoder_model1.5.h5'\nlatent_vectors = np.load('/content/drive/MyDrive/ML final project/Trained_models/encoder/latent_vectors1.5.npy')\nimage_data = np.load('/content/drive/MyDrive/ML final project/Trained_models/encoder/image_data1.5.npy')\nencoder = load_model(filepath)\n#encoder = tf.keras.layers.TFSMLayer('encoder_model', call_endpoint='serving_default')\n#image_data = np.load('/content/drive/MyDrive/ML final project/Trained_models/encoder/encoder_filtered/image_data8.npy')\n"

In [ ]:
# Paths
TEST_IMAGE_FOLDER = '/content/drive/MyDrive/ML final project/datasets/testData'
TEST_LABELS_CSV = '/content/drive/MyDrive/ML final project/datasets/test_labels.csv'
filepath_encoder = '/content/drive/MyDrive/ML final project/Trained_models/encoder/encoder_model_f32'
filepath_decoder = '/content/drive/MyDrive/ML final project/Trained_models/decoder/decoder_model_f32'
filepath_autoencoder = '/content/drive/MyDrive/ML final project/Trained_models/autoencoder_model_f32'

# Load the models (note: no need for custom_objects here)
encoder = tf.keras.models.load_model(filepath_encoder)
decoder = tf.keras.models.load_model(filepath_decoder)
autoencoder = tf.keras.models.load_model(filepath_autoencoder)



In [ ]:
# Load the stored latent vectors
latent_vectors = np.load('/content/drive/MyDrive/ML final project/Trained_models/latent_vectors_f32.npy')

# View a single latent vector (e.g., the first one)
single_vector = latent_vectors[0]  # Gets the first vector
print("Single Latent Vector:")
print(single_vector)

# If you want to see its shape
print("\nVector shape:", single_vector.shape)

# If you want to view a specific index
index = 5  # Change this to view different vectors
specific_vector = latent_vectors[index]
print(f"\nLatent vector at index {index}:")
print(specific_vector)

# Optional: If you want to see it alongside its corresponding image label
# Load the vector mapping file if you saved it
vector_mapping = pd.read_csv('/content/drive/MyDrive/ML final project/Trained_models/vector_mapping_f32.csv')
print(f"\nCorresponding image filename: {vector_mapping['filename'][index]}")
print(f"Eye color label: {vector_mapping['label'][index]}")

Single Latent Vector:
[  3.9139857  -78.696014    40.59769    100.93081      0.30083567
  64.351006   -16.324448     7.0732713  -25.017939    18.731451
 -15.498808    73.33091    -37.94125     36.51313     33.477234
 -17.841972   -25.454254     1.874886    71.60545     14.626062
  31.130869   -33.50864    -48.530712   -74.159355    15.654098
 104.72894    -36.292656   -17.534327    72.05762     -0.9311351
   7.9609513   52.66259   ]

Vector shape: (32,)

Latent vector at index 5:
[ 63.761765  -90.588715   29.164812   34.263607    9.835992   71.956505
  13.545499   64.47211   -33.322178   30.295397  -35.62386     8.381276
 -71.50911    64.85791     4.0841    -43.360874  -35.39441    66.37679
  69.46605    44.815323   -2.2411065 -39.047676  -54.15908   -87.708084
  26.135551   88.97029   -26.898077    9.175446   82.1819     -4.3425303
 -35.106884   49.084946 ]

Corresponding image filename: C1_S1_I6.tiff
Eye color label: brown


# TESTING
function for preprocessing image1 and image2

In [ ]:

def preprocess_image(image):
    """Preprocesses an image for the encoder using PIL for better quality."""
    # Check if image is None
    if image is None:
        raise ValueError("Input image is None")

    # Convert BGR to RGB (OpenCV loads in BGR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert NumPy array to PIL Image
    try:
        image_pil = Image.fromarray(image)
    except Exception as e:
        print(f"Error converting to PIL Image: {e}")
        print(f"Image shape: {image.shape}, dtype: {image.dtype}")
        raise

    # Resize using PIL with high-quality interpolation
    image_resized = image_pil.resize((IMG_SIZE, IMG_SIZE), Image.LANCZOS)

    # Convert back to NumPy array and normalize
    image_resized = np.array(image_resized)

    # Ensure float32 type and proper normalization
    image_normalized = image_resized.astype(np.float32) / 255.0

    # Debug prints
    print(f"Preprocessed image shape: {image_normalized.shape}")
    print(f"Value range: [{image_normalized.min():.3f}, {image_normalized.max():.3f}]")

    return image_normalized

# When loading individual images:
def load_and_preprocess_single_image(image_path):
    """Load and preprocess a single image."""
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not load image from {image_path}")

    # Preprocess
    preprocessed = preprocess_image(image)
    return preprocessed

image1_preprocessed = preprocess_image(image1)
image2_preprocessed = preprocess_image(image2)
image3_preprocessed = preprocess_image(image3)

Preprocessed image shape: (128, 128, 3)
Value range: [0.024, 0.976]
Preprocessed image shape: (128, 128, 3)
Value range: [0.000, 0.984]
Preprocessed image shape: (128, 128, 3)
Value range: [0.016, 0.953]


Function for preprocessing the larger set of test data. (combine with above function?)

In [ ]:
def load_test_data():
    """Load and preprocess test dataset"""
    # Read and sort the CSV file
    labels_df = pd.read_csv(TEST_LABELS_CSV)
    labels_df['C'] = labels_df['filename'].str.extract(r'C(\d+)')[0].astype(int)
    labels_df['S'] = labels_df['filename'].str.extract(r'S(\d+)')[0].astype(int)
    labels_df['I'] = labels_df['filename'].str.extract(r'I(\d+)')[0].astype(int)
    labels_df = labels_df.sort_values(['C', 'S', 'I'])

    # Load and preprocess images
    image_data = []
    for filename in labels_df['filename']:
        image_path = os.path.join(TEST_IMAGE_FOLDER, filename)
        if os.path.exists(image_path):
            img = Image.open(image_path)
            img = img.resize((IMG_SIZE, IMG_SIZE), Image.LANCZOS)
            img_array = np.array(img)
            img_array = img_array.astype("float32") / 255.0
            image_data.append(img_array)

    image_data = np.array(image_data)
    return image_data, labels_df

def generate_latent_vectors(encoder, image_data):
    """Generate latent vectors for the test dataset"""
    return encoder.predict(image_data)

In [ ]:
def preprocess_images(image_path):
    """Preprocesses an image for the encoder using PIL for better quality."""
    input_shape = encoder.input_shape[1:]  # Get input shape from the encoder

    # Open and resize image using PIL with high-quality interpolation
    image_pil = Image.open(image_path)
    image_resized = image_pil.resize((input_shape[1], input_shape[0]), Image.LANCZOS)

    # Convert back to NumPy array and normalize
    image_resized = np.array(image_resized)
    image_normalized = image_resized / 255.0
    image_preprocessed = image_normalized.astype(np.float32)
    return image_preprocessed

def generate_test_data(folder_path):
    """Generates image_data and latent_vectors from a folder of images."""
    image_data = []
    latent_vectors = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.tiff'):  # Adjust extensions as needed
            image_path = os.path.join(folder_path, filename)

            # Load the image first using cv2
            image = cv2.imread(image_path)
            if image is None:
                print(f"Failed to load image: {image_path}")
                continue

            # Convert BGR to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


            # Preprocess the image
            preprocessed_image = preprocess_image(image)

            # Add preprocessed image to image_data
            image_data.append(preprocessed_image)

            # Get latent vector using the encoder
            latent_vector = encoder.predict(np.expand_dims(preprocessed_image, axis=0))
            latent_vectors.append(latent_vector.flatten())  # Flatten to 1D

    return np.array(image_data), np.array(latent_vectors)


# This is used to normalize the embeddings, which he;ps with the decoding process.

In [ ]:


# Normalize the latent vector to range 0 to 1 for better interpretability
def scale_latent_vector(test_latent_vectors):
    min_val = np.min(test_latent_vectors)
    max_val = np.max(test_latent_vectors)

    # Normalize to [0, 1] range
    scaled_latent_vector = (test_latent_vectors - min_val) / (max_val - min_val)
    return scaled_latent_vector


In [ ]:
# WORKING
# USE THOS

# Function to reconstruct images from latent vectors
def reconstruct_images(latent_vectors, num_images=5):
    reconstructed = decoder.predict(latent_vectors[:num_images])
    reconstructed = np.clip(reconstructed, 0, 1)

    # Display original vs reconstructed images
    plt.figure(figsize=(15, 3))
    for i in range(num_images):
        # Original image
        plt.subplot(2, num_images, i + 1)
        plt.imshow(test_image_data[i])
        plt.axis('off')
        if i == 0:
            plt.title('Original')

        # Reconstructed image
        plt.subplot(2, num_images, num_images + i + 1)
        plt.imshow(reconstructed[i])
        plt.axis('off')
        if i == 0:
            plt.title('Reconstructed')

    plt.tight_layout()
    plt.show()

# Function to interpolate between two images
def interpolate_images(decoder, latent_vector1, latent_vector2, num_steps=10):
    """Interpolate between two latent vectors and generate images"""
    alphas = np.linspace(0, 1, num_steps)
    interpolated_vectors = []

    for alpha in alphas:
        interpolated_vector = (1 - alpha) * latent_vector1 + alpha * latent_vector2
        interpolated_vectors.append(interpolated_vector)

    interpolated_vectors = np.array(interpolated_vectors)
    interpolated_images = decoder.predict(interpolated_vectors)

    return interpolated_images

def display_interpolation(interpolated_images, color1=None, color2=None):
    """Display interpolation results"""
    plt.figure(figsize=(15, 3))
    for i in range(len(interpolated_images)):
        plt.subplot(1, len(interpolated_images), i + 1)
        plt.imshow(interpolated_images[i])
        plt.axis('off')
        if i == 0 and color1:
            plt.title(f'{color1}', loc='left')
        elif i == len(interpolated_images)-1 and color2:
            plt.title(f'{color2}', loc='right')
    plt.tight_layout()
    plt.show()

In [ ]:
#print(os.path.exists(filepath))

Functions to interpolate the first image found for each class

In [ ]:
# USE THOS

# WORKING
def get_color_examples(labels_df, latent_vectors, color_name, n=1):
    """Get latent vectors for eyes of a specific color"""
    color_indices = labels_df[labels_df['label'] == color_name].index[:n]
    return latent_vectors[color_indices]

def interpolate_color_sequence(decoder, latent_vectors, labels_df, colors=['brown', 'blue', 'hazel', 'green', 'gray'], steps_between=10):
    """Create a sequence of interpolations between different eye colors"""
    # Get one example for each color
    color_vectors = [get_color_examples(labels_df, latent_vectors, color, 1)[0] for color in colors]

    plt.figure(figsize=(20, 4 * (len(colors)-1)))
    current_plot = 1

    # Interpolate between consecutive colors
    interpolated_images_all = []  # Store all interpolated images
    true_labels_all = []         # Store corresponding labels

    for i in range(len(colors)-1):
        print(f"Interpolating from {colors[i]} to {colors[i+1]}...")

        # Create interpolation
        alphas = np.linspace(0, 1, steps_between)
        interpolated_vectors = []

        for alpha in alphas:
            interpolated_vector = (1 - alpha) * color_vectors[i] + alpha * color_vectors[i+1]
            interpolated_vectors.append(interpolated_vector)

        interpolated_vectors = np.array(interpolated_vectors)
        interpolated_images = decoder.predict(interpolated_vectors)
        interpolated_images = np.clip(interpolated_images, 0, 1)

        interpolated_images_all.extend(interpolated_images)  # Add to the list
        true_labels_all.extend([colors[i]] * steps_between)  # Add corresponding labels

        # Plot this interpolation sequence
        plt.subplot(len(colors)-1, 1, current_plot)
        for j in range(steps_between):
            plt.subplot(len(colors)-1, steps_between, (current_plot-1)*steps_between + j + 1)
            plt.imshow(interpolated_images[j])
            plt.axis('off')
            if j == 0:
                plt.title(f'{colors[i]}', loc='left')
            elif j == steps_between-1:
                plt.title(f'{colors[i+1]}', loc='right')

        current_plot += 1

    plt.tight_layout()
    plt.show()

    return np.array(interpolated_images_all), np.array(true_labels_all) # Return the images and labels

# Function to call decoder to reconstruct images from vectors

In [ ]:
def reconstruct_images(latent_vectors, num_images=5):
    reconstructed = decoder.predict(latent_vectors[:num_images])
    reconstructed = np.clip(reconstructed, 0, 1)

    plt.figure(figsize=(15, 4))
    for i in range(num_images):
        plt.subplot(2, num_images, i + 1)
        plt.imshow(test_image_data[i])
        plt.axis('off')
        if i == 0:
            plt.title('Original')

        plt.subplot(2, num_images, num_images + i + 1)
        plt.imshow(reconstructed[i])
        plt.axis('off')
        if i == 0:
            plt.title('Reconstructed')
        # Calculate Mean Squared Error (MSE)
        mse = np.mean(np.square(test_image_data[:num_images] - reconstructed))
        print(f"Reconstruction MSE: {mse}")

    plt.tight_layout()
    plt.show()

# Function to interpolate two random images from different classes

In [ ]:
import random
# USE THOS

def randomer_interpolater(decoder, latent_vectors, labels_df, colors=['brown', 'blue', 'hazel', 'green', 'gray'], steps_between=10):
    """Create random interpolations between different eye colors"""
    color_vectors = []
    selected_indices = []

    for color in colors:
        color_indices = labels_df[labels_df['label'] == color].index
        if len(color_indices) > 0:
            random_index = random.choice(color_indices)
            color_vectors.append(latent_vectors[random_index])
            selected_indices.append(random_index)
        else:
            print(f"Warning: No images found for color {color}")
            return

    plt.figure(figsize=(20, 5 * (len(colors)-1)))
    current_plot = 1

    for i in range(len(colors)-1):
        print(f"Interpolating from {colors[i]} to {colors[i+1]}...")
        interpolated_images = interpolate_images(
            decoder,
            color_vectors[i],
            color_vectors[i+1],
            steps_between
        )

        # Display original images and interpolation
        for j in range(steps_between):
            plt.subplot(len(colors)-1, steps_between, (current_plot-1)*steps_between + j + 1)
            plt.imshow(interpolated_images[j])
            plt.axis('off')
            if j == 0:
                plt.title(f'{colors[i]}', loc='left')
            elif j == steps_between-1:
                plt.title(f'{colors[i+1]}', loc='right')

        current_plot += 1

    plt.tight_layout()
    plt.show()

# Function to calculate how accurate the interpolation is.
This is a good idea to show evaluation metrics, but it is not currently working

In [ ]:


def calculate_reconstruction_metrics(original_images, reconstructed_images):
    """Calculate various reconstruction error metrics"""
    # Mean Squared Error (MSE)
    mse = mean_squared_error(original_images.reshape(len(original_images), -1),
                           reconstructed_images.reshape(len(reconstructed_images), -1))

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)

    # Mean Absolute Error (MAE)
    mae = np.mean(np.abs(original_images - reconstructed_images))

    # Peak Signal-to-Noise Ratio (PSNR)
    max_pixel = 1.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))

    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'PSNR': psnr
    }

def plot_tsne_visualization(latent_vectors, labels_df, perplexity=30):
    """Create t-SNE visualization of the latent space"""
    # Perform t-SNE dimensionality reduction
    tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42)
    latent_2d = tsne.fit_transform(latent_vectors)

    # Create DataFrame for plotting
    df = pd.DataFrame({
        'x': latent_2d[:, 0],
        'y': latent_2d[:, 1],
        'color': labels_df['label']
    })

    # Plot
    plt.figure(figsize=(10, 8))
    sns.scatterplot(data=df, x='x', y='y', hue='color', alpha=0.6)
    plt.title('t-SNE Visualization of Latent Space')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.legend(title='Eye Color')
    plt.show()



def calculate_interpolation_accuracy(interpolated_images, true_labels, color_mapping):
    predicted_labels = []
    for image in interpolated_images:
        dominant_color = get_dominant_iris_color(image)
        if dominant_color is not None:
            min_dist = float('inf')
            predicted_color = None

            for color_name, color_value in color_mapping.items():
                color1_rgb = sRGBColor(*dominant_color / 255.0)
                color2_rgb = sRGBColor(*np.array(color_value) / 255.0)
                color1_lab = convert_color(color1_rgb, LabColor)
                color2_lab = convert_color(color2_rgb, LabColor)

                delta_e = delta_e_cie2000(color1_lab, color2_lab)

                if delta_e < min_dist:
                    min_dist = delta_e
                    predicted_color = color_name

            predicted_labels.append(predicted_color)
        else:
            predicted_labels.append(None)

    # Filter out None values (cases where iris was not detected)
    valid_indices = [i for i, label in enumerate(predicted_labels) if label is not None]
    predicted_labels = np.array(predicted_labels)[valid_indices]
    true_labels = true_labels[valid_indices]

    # Calculate accuracy
    accuracy = np.mean(predicted_labels == true_labels)
    print(f"Interpolation Accuracy: {accuracy}")
    return accuracy


import umap
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def plot_multiple_visualizations(latent_vectors, labels_df):
    # Set up the figure
    fig, axes = plt.subplots(2, 2, figsize=(20, 20))

    # 1. UMAP
    reducer = umap.UMAP(random_state=42)
    umap_embedded = reducer.fit_transform(latent_vectors)

    sns.scatterplot(
        x=umap_embedded[:, 0],
        y=umap_embedded[:, 1],
        hue=labels_df['label'],
        alpha=0.6,
        ax=axes[0,0]
    )
    axes[0,0].set_title('UMAP Visualization')

    # 2. PCA
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(latent_vectors)

    sns.scatterplot(
        x=pca_result[:, 0],
        y=pca_result[:, 1],
        hue=labels_df['label'],
        alpha=0.6,
        ax=axes[0,1]
    )
    axes[0,1].set_title(f'PCA Visualization\nExplained variance: {pca.explained_variance_ratio_.sum():.2%}')

    # 3. Distribution of latent dimensions
    # Take first 10 dimensions for visualization

    latent_df = pd.DataFrame(latent_vectors[:, :10],
                           columns=[f'Dim_{i+1}' for i in range(10)])

    sns.boxplot(data=latent_df, ax=axes[1,0])
    axes[1,0].set_title('Distribution of 8 Latent Dimensions')
    axes[1,0].tick_params(axis='x', rotation=45)

    # 4. Correlation matrix of first 10 latent dimensions
    corr_matrix = latent_df.iloc[:, :10].corr()
    sns.heatmap(corr_matrix, ax=axes[1,1], cmap='coolwarm', center=0)
    axes[1,1].set_title('Correlation Matrix of Latent Dimensions')

    plt.tight_layout()
    plt.show()

    # Additional: Class distribution
    plt.figure(figsize=(10, 5))
    sns.countplot(data=labels_df, x='label')
    plt.title('Distribution of Eye Colors in Dataset')
    plt.xticks(rotation=45)
    plt.show()




# Function to plot 20 random images
This function needs some tweaking, but it is great for visualizing the dataset, and ensuring that labels are attached to the appropriate image.

In [ ]:
# TESTING
def plot_random_samples(test_data, idx_to_color):
    """Plot 20 random samples from the test data in a 4x5 grid"""
    # Select 20 random indices
    indices = np.random.choice(len(test_data), 20, replace=False)

    plt.figure(figsize=(15, 12))
    for i, idx in enumerate(indices):
        # Get image and label
        image, label = test_data[idx]

        # Denormalize the image
        mean = torch.tensor([0.485, 0.456, 0.406]).reshape(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).reshape(3, 1, 1)
        img_denorm = image * std + mean

        # Plot
        plt.subplot(4, 5, i + 1)
        plt.imshow(img_denorm.permute(1, 2, 0).clamp(0, 1))
        plt.title(idx_to_color[label.item()], pad=5)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Main function

explaination:
- loads the label csv file. If you use other dataset, change this
- color mapping: This is used for any future plotting, so that points can be colored according to their color class.
- reconstruct images: displays the first five images, then reconstructs them via the decoder.
- interpolating: there are two function calls, and one is commented out. If you want to interpolate from the same images each time you run this main, uncomment the first function call, and comment out the randomer_interpolater function. If you want to see random image interpolation, do the opposite.


In [ ]:
    print("Loading test data...")
    test_image_data, test_labels_df = load_test_data()
    print(f"Loaded {len(test_image_data)} test images")
    # Generate latent vectors for test data
    print("Generating latent vectors...")
    test_latent_vectors = generate_latent_vectors(encoder, test_image_data)

In [ ]:
labels_df.head()

In [ ]:
# prompt: Using dataframe labels_df: print the images of the eyes        1238, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383,

# Convert the string of image indices to a list of integers.
image_indices = [1238, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383]

# Assuming the filenames are in sequential order in the dataframe.
# Extract the filenames for the specified image indices from the dataframe.

if not all(0 <= i < len(labels_df) for i in image_indices):
    raise IndexError("Image indices are out of the valid range.")

# Extract filenames.
filenames_to_print = labels_df.iloc[image_indices]['filename'].tolist()

# Now you can use filenames_to_print to access and print the images.
# Note: This is a placeholder; you'll need to provide the code to load and display the images.
for filename in filenames_to_print:
    print(f"Displaying image: {filename}")
    # Add your image loading and display code here.  For example:
    # from PIL import Image
    image = Image.open(f"/content/drive/MyDrive/ML final project/datasets/EYE_IMAGES_FULL/EYE_IMAGES_FULL/{filename}")
    image.show()


In [ ]:
# USE THOS

if __name__ == "__main__":
    # Load the labels
    labels_df = pd.read_csv('/content/drive/MyDrive/ML final project/datasets/test_labels.csv')

    color_mapping = {
        'brown': [165, 42, 42],  # Example RGB for brown
        'blue': [0, 0, 255],    # Example RGB for blue
        'hazel': [184, 134, 11],  # Example RGB for hazel
        'green': [0, 128, 0]   # Example RGB for green
    }

    print("Reconstructing sample images...")
    reconstruct_images(test_latent_vectors)

    print("\nInterpolating between different eye colors...")
    interpolate_color_sequence(decoder, latent_vectors, labels_df)
    randomer_interpolater(decoder, test_latent_vectors, test_labels_df)

    interpolated_images, true_labels = interpolate_color_sequence(decoder, latent_vectors, labels_df)
    #interpolation_accuracy = calculate_interpolation_accuracy(interpolated_images, true_labels, color_mapping)

    #print("\nFinding best representative images for each color class...")
   # find_best_representative_images(image_data, labels_df)

    #print("\nExperimenting with hue...")
    #interpolate_and_apply_color_from_classes(find_best_representative_images(image_data, labels_df), image_data, labels_df)

    # Calculate reconstruction metrics
    print("\nCalculating reconstruction metrics...")
    reconstructed_images = decoder.predict(test_latent_vectors)
    metrics = calculate_reconstruction_metrics(test_image_data, reconstructed_images)

    print("\nReconstruction Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

    # Visualize latent space using t-SNE
    print("\nGenerating t-SNE visualization...")
    plot_tsne_visualization(test_latent_vectors, test_labels_df)

    # Add to your main section:
    print("\nGenerating additional visualizations...")
    plot_multiple_visualizations(test_latent_vectors, test_labels_df)

# Result of interpolate_color_sequence(decoder, latent_vectors, labels_df)
# NOTE: These results were also using the filtered iris dataset. If you want to reproduce this, you must load the proper dataset.

In [ ]:
def interpolate_two_images(latent_vector1, latent_vector2, num_steps=10):
    alphas = np.linspace(0, 1, num_steps)
    interpolated_vectors = []

    for alpha in alphas:
        interpolated_vector = (1 - alpha) * latent_vector1 + alpha * latent_vector2
        interpolated_vectors.append(interpolated_vector)

    # Reshape the interpolated_vectors to (num_steps, 64)
    interpolated_vectors = np.array(interpolated_vectors).reshape(num_steps, latent_vector1.shape[-1]) # Reshape to match decoder input
    interpolated_images = decoder.predict(interpolated_vectors)

    # Display interpolation results
    plt.figure(figsize=(15, 3))
    for i in range(num_steps):
        plt.subplot(1, num_steps, i + 1)
        plt.imshow(interpolated_images[i])
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# TESTING GROUP MEMBER EYES

In [ ]:
latent_vector1 = encoder.predict(np.expand_dims(image1_preprocessed, axis=0))
latent_vector2 = encoder.predict(np.expand_dims(image2_preprocessed, axis=0))
latent_vector3 = encoder.predict(np.expand_dims(image3_preprocessed, axis=0))


interpolate_two_images(latent_vector1, latent_vector2, num_steps=10)

# GRADIO TESTING

In [ ]:


def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["image"],
    outputs=["text"],
)



demo.launch()

In [ ]:
def create_interpolation(img1, img2):
    """Handle the Gradio interface and create interpolation"""
    try:
        # Directly normalize the input images without color space conversion
        img1_preprocessed = np.array(img1).astype(np.float32) / 255.0
        img2_preprocessed = np.array(img2).astype(np.float32) / 255.0

        # Resize if needed
        if img1_preprocessed.shape[:2] != (IMG_SIZE, IMG_SIZE):
            img1_preprocessed = cv2.resize(img1_preprocessed, (IMG_SIZE, IMG_SIZE))
        if img2_preprocessed.shape[:2] != (IMG_SIZE, IMG_SIZE):
            img2_preprocessed = cv2.resize(img2_preprocessed, (IMG_SIZE, IMG_SIZE))

        # Get latent vectors
        latent1 = encoder.predict(np.expand_dims(img1_preprocessed, axis=0))
        latent2 = encoder.predict(np.expand_dims(img2_preprocessed, axis=0))

        # Create interpolation vectors
        num_steps = 10
        alphas = np.linspace(0, 1, num_steps)
        interpolated_vectors = [(1 - alpha) * latent1[0] + alpha * latent2[0] for alpha in alphas]

        # Generate images from interpolated vectors
        interpolated_images = decoder.predict(np.array(interpolated_vectors))

        # Post-process images
        interpolated_images = np.clip(interpolated_images * 255, 0, 255).astype(np.uint8)

        # Create GIF with forward and backward interpolation for smooth loop
        frames = list(interpolated_images) + list(interpolated_images[::-1])

        # Create GIF that loops 10 times
        output_path = "interpolation.gif"
        imageio.mimsave(output_path, frames, duration=200, loop=10)

        return output_path

    except Exception as e:
        print(f"Error: {str(e)}")
        raise gr.Error(str(e))


# Create two tabs: one for file upload and one for webcam
with gr.Blocks() as demo:
    gr.Markdown("# Eye Color Interpolation")
    gr.Markdown("Upload two eye images or use webcam to capture images and see the interpolation between them.")

    with gr.Tab("Upload Images"):
        with gr.Row():
            image1_upload = gr.Image(type="numpy", label="First Image")
            image2_upload = gr.Image(type="numpy", label="Second Image")
        upload_button = gr.Button("Create Interpolation")
        output_upload = gr.Image(type="filepath", label="Interpolation Result")
        upload_button.click(
            create_interpolation,
            inputs=[image1_upload, image2_upload],
            outputs=output_upload
        )

    with gr.Tab("Webcam"):
        with gr.Row():
            image1_webcam = gr.Image(type="numpy", label="First Image", sources="webcam")
            image2_webcam = gr.Image(type="numpy", label="Second Image", sources="webcam")
        webcam_button = gr.Button("Create Interpolation")
        output_webcam = gr.Image(type="filepath", label="Interpolation Result")
        webcam_button.click(
            create_interpolation,
            inputs=[image1_webcam, image2_webcam],
            outputs=output_webcam
        )

# Launch the interface
demo.launch(debug=True)